In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

from datetime import datetime 
import schedule
import time

In [2]:
# usuario = 
# contraseña =  
# mt5.login(usuario, contraseña)
if not mt5.initialize():
    print('Error al iniciar: ',mt5.last_error())
    quit()
else:
    print('Inicio correctamente')

Inicio correctamente


# Importando Datos MT5

https://www.mql5.com/es/docs/python_metatrader5

In [3]:
def Simbolo(divisa, tf,em,ter):  
           
    temp = {'M1':mt5.TIMEFRAME_M1,'M5':mt5.TIMEFRAME_M5,
            'M15':mt5.TIMEFRAME_M15,'M30':mt5.TIMEFRAME_M30,
            'H1':mt5.TIMEFRAME_H1,'H4':mt5.TIMEFRAME_H4
           }
    for clave, cont in temp.items():
        if tf == clave:
            rates = mt5.copy_rates_from_pos(divisa, cont, em, ter)

    # Transform Tuple into a DataFrame
    df_rates = pd.DataFrame(rates).dropna()
    

    # Convert number format of the date into date format
    df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
    df_rates = df_rates.drop(['tick_volume','spread','real_volume'],axis=1)
    
    df_rates.time = pd.to_datetime(df_rates['time'])
    df_rates.index = df_rates.time
    df_rates.index.name = 'date'
    df_rates['time'] = df_rates['time'].apply(mpl_dates.date2num)

    return df_rates

symbol    = 'EURUSD' 
timeframe = 'M5'
ini = 0
ter = 180
df = Simbolo(symbol,timeframe,ini,ter)
# df.head()

# SuperTrend (ATR)

https://medium.datadriveninvestor.com/the-supertrend-implementing-screening-backtesting-in-python-70e8f88f383d

In [4]:
def SuperTrend(df):
    #ATR
    high_low = np.abs(df['high'] - df['low'])
    high_pc = np.abs(df['high'] - df['close'].shift())
    low_pc = np.abs(df['low'] - df['close'].shift())

    ranges = pd.concat([high_low, high_pc, low_pc],axis=1)
    true_range = np.max(ranges, axis=1)
    df['atr'] = true_range.ewm(alpha=1/14,min_periods=14).mean() #5 14
    
    multiplier = 2.5
    df['upperband'] = ((df.high + df.low) / 2) + (multiplier * df.atr)
    df['lowerband'] = ((df.high + df.low) / 2) - (multiplier * df.atr)
    df['SuperTrend'] = True

    for actual in range(1, len(df.index)):
        anterior = actual - 1

        if df['close'][actual] > df['upperband'][anterior]:
            df['SuperTrend'][actual] = True
        elif df['close'][actual] < df['lowerband'][anterior]:
            df['SuperTrend'][actual] = False
        else:
            df['SuperTrend'][actual] = df['SuperTrend'][anterior]

            if df['SuperTrend'][actual] == True and df['lowerband'][actual] < df['lowerband'][anterior]:
                df['lowerband'][actual] = df['lowerband'][anterior]

            if df['SuperTrend'][actual] == False and df['upperband'][actual] > df['upperband'][anterior]:
                df['upperband'][actual] = df['upperband'][anterior]
        
        if df.SuperTrend[actual] == True:
            df.upperband[actual] = np.nan
        else:
            df.lowerband[actual] = np.nan
            
    return df

st = SuperTrend(df)
# st

# Velas Heikin Ashi

In [5]:
def  Heikin_Ashi(df):
    dt = pd.DataFrame(index=df.index)
    dt['close'] = (df['high'] + df['low'] + df['open'] + df['close']) / 4
    dt['time'] = df['time']
    dt['open'] = df['open']

    for actual in range(1,len(df.index)):
        anterior = actual - 1
        dt['open'][actual] = (df.open[anterior] + df.close[anterior]) / 2

    dt['high'] = df['high']
    dt['high'] = dt[['high','open','close']].max(axis=1)
    dt['low']  = df['low']
    dt['low']  = dt[['low','open','close']].min(axis=1)        
    return dt

dt = Heikin_Ashi(df)
# dt

#  Manejo de ordenes

* Documentacion:https://www.mql5.com/es/docs/python_metatrader5

In [7]:
def orders(symbol, lot, buy=False, id_position=None):
    
    """ Send the orders """

    filling_mode = mt5.symbol_info(symbol).filling_mode
    deviation = 10
    
    # **************************** Open a trade *****************************       
    # Buy order Parameters
    if buy:
        type_trade = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
        sl = price - 50 * mt5.symbol_info(symbol).point
#             sl = mt5.symbol_info_tick(symbol).ask * (100-0.01)
        tp = price + 100 * mt5.symbol_info(symbol).point 
#             tp = mt5.symbol_info_tick(symbol).ask * (300+0.01)

    # Sell order Parameters
    else:            
        type_trade = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
        sl = price + 50 * mt5.symbol_info(symbol).point
#             sl = price * (1+0.01)
        tp = price - 100 * mt5.symbol_info(symbol).point 
#             tp = price * (3-0.01)

    # Open the trade
    request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": type_trade,
                "price": price,
                "deviation": deviation,
                "sl": sl,
                "tp": tp,
                "magic": 234000,
                "comment": "python script order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": filling_mode,
                }
    
    # send a trading request
    result = mt5.order_send(request)
    
    return result.comment

In [13]:
posicion = False 

def señales_compra_o_venta(df, symbol):
    global posicion
      
    print('buscando señales..........')
    
    actual    = len(df.index) - 1
    anterior = actual - 1
    
    if df['SuperTrend'][anterior] == True and df['SuperTrend'][actual]==False:
        print('Cambio de tendencia, VENTAS','\n')
        if not posicion:
            print(orders(symbol, 0.01, False))
            posicion = True
        else:
            print('Ya se encuentra en posicion de VENTAS')
           
    if df['SuperTrend'][anterior] == False and df['SuperTrend'][actual] == True:
        print('Cambio de tendencia, COMPRAS','\n')
        if posicion:
            print(orders(symbol, 0.01, True))
            posicion = False
        else:
            print('Ya se encuentra en posicion de COMPRAS')

# Backtesting


In [ ]:
def backtest_supertrend(data, investment):
    is_uptrend = data['SuperTrend']
    close = data['close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(data)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = math.floor(equity / close[i] / 100) * 10
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} shares at {round(close[i],2)} on {data.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell at {round(close[i],2)} on {data.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100 is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity

In [ ]:
# velas normales
entry, exit, roi = backtest_supertrend(df, 100)

print(entry, exit, roi, sep='\n\n')

In [ ]:
# velas Heikin Ashi
entry, exit, roi = backtest_supertrend(velas_HA, 100)

print(entry, exit, roi, sep='\n\n')

# Bot de negociacion

In [14]:
def run_bot():
    data   = Simbolo(symbol,timeframe,0,50)
    print(f'Buscando barras nuevas desde: {data.index[-1]}','\n')
    velas_HA = Heikin_Ashi(data)
    STdata = SuperTrend(velas_HA)
    señales_compra_o_venta(STdata, symbol)

schedule.every(10).seconds.do(run_bot)

while True:
    schedule.run_pending()
    
    time.sleep(1) 
    
# schedule.every(10).seconds.do(run_bot)
# start = datetime.now().strftime("%H:%M:%S") #"23:59:59"

# while True:
#     if datetime.now().weekday() not in (5,6): # If you want to trade only from Monday to Friday
#         is_time = datetime.now().strftime("%H:%M:%S") == start 
#     else:
#         is_time = False
    
#     # Launch the algorithm
#     if is_time:
#         schedule.run_pending()
    
#         time.sleep(1)     

Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 17:50:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nue

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:00:00 

buscando señales..........
buscando señales..........
buscando señales...

Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:05:00 

buscando señales.....

buscando señales..........
buscando señales..........
Buscando barras nuevas desde: 2023-05-16 18:10:00 

buscando señales..........


KeyboardInterrupt: 

In [ ]:
# Create the bound between MT5 and Python
mt5.shutdown()